# Horoscope Generator

Train a machine learning model and generate horoscopes based on dataset from horoscope.com

I followed this guide: https://www.kaggle.com/code/shivamb/beginners-guide-to-text-generation-using-lstms

This generator notebook will create models for each of the zodiac signs. 

First import modules:

In [1]:
import tensorflow as tf

# keras module for building LSTM 
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import utils as ku

# set seeds for reproducability
from numpy.random import seed
tf.random.set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

tf.__version__


'2.10.0'

# Import the dataset:

Read all of the json files for each horoscope sign

In [ ]:

import pandas as pd
import re

signNames = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]

megaCorpus = {}

for i in range(1, 13): # exclude aries since I've already created and saved a model for aries
    df = pd.read_json('dataset2/'+ signNames[i-1] +'.json')
    corpus = list(df["text"])
    megaCorpus[signNames[i-1]] = corpus
    inp_sequences, total_words = get_sequence_of_tokens(corpus)
    print(total_words)

print(megaCorpus.keys())

# Define functions for sequences, creating the model and generating text

In [2]:
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words, tokenizer

def generate_padded_sequences(input_sequences, total_words):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+ output_word
    return seed_text.title()

# Train the Models

In [8]:
generated_horoscopes = {}

def train_models():
    
    for i in range(10, 13):
        KBE.clear_session()
        
        tokenizer = Tokenizer()
            
        curr_sign = signNames[i-1]
        print(curr_sign)
        
        df = pd.read_json('dataset2/'+ curr_sign +'.json')
        corpus = list(df["text"])

        # generate sequences
        inp_sequences, total_words, tokenizer = get_sequence_of_tokens(corpus)
        print(total_words)

        # obtain predictors and label
        predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences, total_words)

        # create models
        model = create_model(max_sequence_len, total_words)
        model.summary()

        print('--')

        # train models
        model.fit(predictors, label, epochs=100, verbose = 5)

        print('--')
        # evaluate model
        results = model.evaluate(x=predictors, y=label)
        print(results)

        print('--')

        model.save("model2-" + signNames[i-1] + ".h5")
        print("model saved")

        print('--')

        generated_text = generate_text("today", 100, model, max_sequence_len, tokenizer)
        generated_horoscopes[curr_sign] = generated_text
        print(generated_text)



# Generating text with the saved models

In [5]:
from keras import models as KM
from keras import backend as KBE

def horoscope_generator():
    signNames = ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", 
             "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]

    generated_horoscopes = {}

    for i in range(1, 13):
        KBE.clear_session()
        
        
        curr_sign = signNames[i-1]
        
        print(curr_sign)
        
        df = pd.read_json('dataset2/'+ curr_sign +'.json')
        corpus = list(df["text"])

        # generate sequences
        inp_sequences, total_words, tokenizer = get_sequence_of_tokens(corpus)
        print(total_words)

        # obtain predictors and label
        predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences, total_words)

        print('--')

        # load models
        model = KM.load_model("Models/model2-" + curr_sign + ".h5")
        print("model loaded: " + "model2-" + curr_sign + ".h5")
        # model.summary()

        print('--')

        generated_text = generate_text("today", 100, model, max_sequence_len, tokenizer)
        generated_horoscopes[curr_sign] = generated_text
        
    for i in range(1, 13): 
        print("Generated horoscope for " + signNames[i-1] + ": ")
        print(generated_horoscopes[signNames[i-1]])
        print('--')



In [6]:
horoscope_generator()

Aries
2714
--
Metal device set to: Apple M1


2022-10-03 11:04:15.156714: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-03 11:04:15.157181: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


model loaded: model2-Aries.h5
--


2022-10-03 11:04:15.454252: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-03 11:04:15.619490: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 11:04:15.674408: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Taurus
2685
--
model loaded: model2-Taurus.h5
--


...

Aquarius
2688
--
model loaded: model2-Aquarius.h5
--


2022-10-03 11:04:48.757015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 11:04:48.789888: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Pisces
2670
--
model loaded: model2-Pisces.h5
--


2022-10-03 11:04:52.123454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 11:04:52.155646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Generated horoscope for Aries: 
Today Is A Terrific Day For You Aries You Aries You Might Find Upsetting Bear In This So Youre Important That Youll Probably Find Yourself Much More Of You Could Surprise You If Youre Surrounded Of Your Destiny To Your Situation Should Be Behind You Today Aries Busy And Bad On A While By Going On A Quick Day To Commit You Your Emotions And Ask Only To You Realize Of Electrical Fingertips Today Could Be A Critical Element To Healing But Your Partner Is The Most Or Sign And Lunch In The Matter You Can Work Out Of Warm And
--
Generated horoscope for Taurus: 
Today You Might Find Yourself Feeling A Bit High Taurus You Today And Youll Find Yourself The Center Of Attention Taurus Perhaps You Feel In Your Emotions And Take Care Of The People You Have Even Much More Information At The Afternoon It Is No Real Count At A New Project At You Youll Be Difficult To Say Where You Might Be Tempted To Lie From The Desire To Pursue The Leap In The Equation The Solution Wi

In [10]:
train_models()

Capricorn
2590
Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________


2022-10-03 08:40:39.289972: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-03 08:40:39.290550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 89, 10)            25900     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2590)              261590    
                                                                 
Total params: 331,890
Trainable params: 331,890
Non-trainable params: 0
_________________________________________________________________
--
Epoch 1/100


2022-10-03 08:40:39.651607: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-03 08:40:40.154238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 08:40:40.269196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 08:40:40.492760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78/100
Epoch 7

...

2022-10-03 09:27:41.552822: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 09:27:41.588185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Today You Might Feel A Bit Restless Aquarius Perhaps A Romantic Partner If You Need To Tend To Step A Leadership Path Or Tasks You Need To Anything Down Try To Say With It All The Opposite Sex You Have To A Project That You Can Take The Evening Remember That Youre Probably Romantically Really An Intelligent Person Who Could Be Involved Ahead And Focused And Others Should Be Hardpressed To Make Lists And Cross In The Promise Of Quick Riches Any Deal That Youll Turn Out Of It Make Your Evening Dont Let You Throw A Bit Until You Need
Pisces
2670
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 90, 10)            26700     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None,

2022-10-03 09:27:45.432663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 09:27:45.535767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-03 09:27:45.739844: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100


KeyboardInterrupt: 